In [1]:
pip install faker


   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---- ----------------------------------- 0.2/1.7 MB 3.9 MB/s eta 0:00:01
   ----- ---------------------------------- 0.2/1.7 MB 2.4 MB/s eta 0:00:01
   ------ --------------------------------- 0.3/1.7 MB 1.6 MB/s eta 0:00:01
   ------- -------------------------------- 0.3/1.7 MB 1.3 MB/s eta 0:00:02
   -------- ------------------------------- 0.4/1.7 MB 1.2 MB/s eta 0:00:02
   --------- ------------------------------ 0.4/1.7 MB 1.1 MB/s eta 0:00:02
   ---------- ----------------------------- 0.5/1.7 MB 1.1 MB/s eta 0:00:02
   ----------- ---------------------------- 0.5/1.7 MB 1.0 MB/s eta 0:00:02
   ------------ --------------------------- 0.5/1.7 MB 964.2 kB/s eta 0:00:02
   ------------ --------------------------- 0.5/1.7 MB 903.5 kB/s eta 0:00:02
   ------------ --------------------------- 0.5/1.7 MB 903.5 kB/s eta 0:00:02
   ------------ --------------------------- 0.5/1.7 MB 903.5 kB/s eta 0:00:02
   --------

In [38]:
pip install sqlparse

     ---------------------------------------- 0.0/41.2 kB ? eta -:--:--
     ---------------------------------------  41.0/41.2 kB ? eta -:--:--
     ---------------------------------------  41.0/41.2 kB ? eta -:--:--
     ---------------------------------------  41.0/41.2 kB ? eta -:--:--
     -------------------------------------- 41.2/41.2 kB 152.4 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [82]:
pip install PrettyTable


Note: you may need to restart the kernel to use updated packages.


In [97]:
pip install pandas sqlite3


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [214]:
import csv
import sqlite3
from prettytable import PrettyTable 

def display_table(data, headers):
    if not data:
        print("No records found.")
        return

    table = PrettyTable(headers)

    for row in data:
        table.add_row(row)

    print(table)

# Problem 1: Read phone book records from CSV
def read_csv(file_path):
    records = []
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            records.append(row)
    return records

# Problem 2: Implement SQL-like parser for CRUD operations
class PhoneBookManager:
    def __init__(self, database_name):
        self.connection = sqlite3.connect(database_name)
        self.cursor = self.connection.cursor()
        self.create_table()

    def create_table(self):
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS phone_records (
                Name TEXT,
                Email TEXT,
                Phone1 TEXT,
                Phone2 TEXT
            )
        ''')
        self.connection.commit()

    def execute_query(self, query):
        try:
            self.cursor.execute(query)
            self.connection.commit()
            return True
        except sqlite3.Error as e:
            print(f"Error executing query: {e}")
            return False

    def select_records(self, condition=None):
        query = 'SELECT * FROM phone_records'
        if condition:
            query += f' WHERE {condition}'
        return self.cursor.execute(query).fetchall()

    def insert_record(self, values):
        query = f"INSERT INTO phone_records(Name, Email, Phone1, Phone2) VALUES {values}"
        return self.execute_query(query)

    def delete_record(self, condition):
        query = f"DELETE FROM phone_records WHERE {condition}"
        return self.execute_query(query)

# Usage
if __name__ == "__main__":
    # Problem 1
    phone_records = read_csv('phone_records1.csv')

    # Problem 2
    phone_book_manager = PhoneBookManager('phone_books1.db')

    # Insert records into the database
    existing_names = set()  # To track existing names and avoid duplicates
    for record in phone_records:
        name = record.get('Name', '').strip()
        email = record.get('Email', '')
        phone1 = record.get('Phone1', '')
        phone2 = record.get('Phone2', '')
        
        # Check for duplicate names and skip insertion if already exists
        if name.lower() not in existing_names:
            values = f"('{name}', '{email}', '{phone1}', '{phone2}')"
            phone_book_manager.insert_record(values)
            existing_names.add(name.lower())

    # SELECT * FROM phone_records
    print("\n2.1 SELECT * FROM phone_records")
    results = phone_book_manager.select_records()
    headers = [desc[0] for desc in phone_book_manager.cursor.description]
    display_table(results, headers)

    # SELECT * FROM phone_records WHERE name='Doe'
    print("\n2.2 SELECT * FROM phone_records WHERE name='Doe'")
    results_doe = phone_book_manager.select_records("Name='Doe'")
    display_table(results_doe, headers)

    # INSERT INTO phone_records
    print("\n2.3 INSERT INTO phone_records")
    new_record_values = "('Test', 'test@test.xtyz', '1234456', '1233233')"
    phone_book_manager.insert_record(new_record_values)

    # SELECT * FROM phone_records after insert
    print("\n2.1 SELECT * FROM phone_records")
    results_after_insert = phone_book_manager.select_records()
    display_table(results_after_insert, headers)

    print("\n2.4 DELETE FROM phone_records WHERE LOWER(Name)='john'")
    phone_book_manager.delete_record("LOWER(Name)='john'")

    # SELECT * FROM phone_records after delete
    print("\n2.1 SELECT * FROM phone_records")
    results_after_delete = phone_book_manager.select_records()
    display_table(results_after_delete, headers)

    phone_book_manager.connection.close()



2.1 SELECT * FROM phone_records
+-------+--------------------+------------+------------+
|  Name |       Email        |   Phone1   |   Phone2   |
+-------+--------------------+------------+------------+
|  John | John123@gmail.com  | 3011276733 |  51413683  |
|  Eric | Eric345@gmail.com  | 807907572  | 7554296020 |
|  Doe  |  Doe234@gmail.com  | 2758705568 | 8319638244 |
|  Will | Will565@yahoo.com  | 8223003267 | 9299399335 |
|  Qery |   Qery@email.com   | 6921864180 | 604475547  |
|  Sam  |  Sam87@yahoo.com   | 8449020590 | 6260260918 |
|  Mady | Mady8909@gmail.com | 8106230888 | 183258841  |
|  Rev  |  Rev67@email.com   | 5397052241 | 7548905021 |
|  Zen  |  Zen675@gamil.com  | 967690217  | 5381557406 |
| Harry | harr678y@gmail.com | 2334544554 |  23456789  |
+-------+--------------------+------------+------------+

2.2 SELECT * FROM phone_records WHERE name='Doe'
+------+------------------+------------+------------+
| Name |      Email       |   Phone1   |   Phone2   |
+------+---